In [2]:
import psaw 
from psaw import PushshiftAPI
import pandas as pd
import numpy as np
from textblob import TextBlob
import praw
import datetime as dt
# api = PushshiftAPI()
# gen = api.search_submissions(filter=['url','author', 'title', 'subreddit', "body"] ,url= "https://www.reddit.com/r/croatia/comments/ydth5v/pa_ti_radi_i_%C5%BEivi_u_glavnom_gradu/")
# thing = list(gen)
# print(thing)

In [44]:
reddit = praw.Reddit(client_id='gP1ZDwru6lQ7_Q', \
                     client_secret='', \
                     user_agent='Scraping', \
                     username='', \
                     password='')

submission = reddit.submission("yd4031")



In [45]:
from praw.models import MoreComments

from flair.models import TextClassifier
from flair.data import Sentence


# for top_level_comment in submission.comments:
#     if isinstance(top_level_comment, MoreComments):
#         continue
#     print(top_level_comment.body)




Dict = { "body":[] }


allcoms = []
submission.comments.replace_more(limit=1200)
for top_level_comment in submission.comments:
    Dict["body"].append(top_level_comment.body)
    

df = pd.DataFrame.from_dict(Dict, orient='index')
df = df.transpose()
# for comment in top_level_comment.body:
#     classifier.predict(Sentence(comment))
#     Dict["score"].append(Sentence.labels)                       
df

,body,score
0,"\nAs a reminder, this subreddit [is for civil ...",None
1,I will be the first to say we need another mou...,None
2,Stop giving companies rights.,None
3,Those who make peaceful revolution impossible ...,None
4,Tell us you want slavery without calling us sl...,None
...,...,...
354,And still millions of clueless people believe ...,None
355,I live in the area. 174’s strikes have far rea...,None
356,And this is what people want.,None
357,We have freedom in this country one of them is...,None


In [4]:
# classifier = TextClassifier.load('en-sentiment')
# sentence = Sentence('The food was great!')
# classifier.predict(sentence)

# print(sentence.labels)

In [46]:
from flair.models import TextClassifier
from flair.data import Sentence
from segtok.segmenter import split_single
import pandas as pd
import re

classifier = TextClassifier.load('en-sentiment')



2022-10-26 23:27:17,478 loading file C:\Users\Van Cloui\.flair\models\sentiment-en-mix-distillbert_4.pt


In [21]:
#ZA CISTIT FUNKCIJE 

def clean(raw):
    """ Remove hyperlinks and markup """
    result = re.sub("<[a][^>]*>(.+?)</[a]>", 'Link.', raw)
    result = re.sub('&gt;', "", result)
    result = re.sub('&#x27;', "'", result)
    result = re.sub('&quot;', '"', result)
    result = re.sub('&#x2F;', ' ', result)
    result = re.sub('<p>', ' ', result)
    result = re.sub('</i>', '', result)
    result = re.sub('&#62;', '', result)
    result = re.sub('<i>', ' ', result)
    result = re.sub("\n", '', result)
    return result

def make_sentences(text):
    """ Break apart text into a list of sentences """
    sentences = [sent for sent in split_single(text)]
    return sentences

def predict(sentence):
    """ Predict the sentiment of a sentence """
    if sentence == "":
        return 0
    text = Sentence(sentence)
#     stacked_embeddings.embed(text)
    classifier.predict(text)
    value = text.labels[0].to_dict()['value'] 
    if value == 'POSITIVE':
        result = text.to_dict()['labels'][0]['confidence']
    else:
        result = -(text.to_dict()['labels'][0]['confidence'])
    return round(result, 3)

def get_scores(sentences):
    """ Call predict on every sentence of a text """
    results = []
    
    for i in range(0, len(sentences)): 
        results.append(predict(sentences[i]))
    return results

def get_sum(scores):
    
    result = round(sum(scores), 3)
    return result

In [50]:


#Clean 

df.body = df.body.apply(clean)
df['sentences'] = df.body.apply(make_sentences)
# df['scores'] = df['sentences'].apply(get_scores)
# df['scores_sum'] = df.scores.apply(get_sum)

df

,body,score,sentences
0,"As a reminder, this subreddit [is for civil di...",None,"[As a reminder, this subreddit [is for civil d..."
1,I will be the first to say we need another mou...,None,[I will be the first to say we need another mo...
2,Stop giving companies rights.,None,[Stop giving companies rights.]
3,Those who make peaceful revolution impossible ...,None,[Those who make peaceful revolution impossible...
4,Tell us you want slavery without calling us sl...,None,[Tell us you want slavery without calling us s...
...,...,...,...
354,And still millions of clueless people believe ...,None,[And still millions of clueless people believe...
355,I live in the area. 174’s strikes have far rea...,None,"[I live in the area., 174’s strikes have far r..."
356,And this is what people want.,None,[And this is what people want.]
357,We have freedom in this country one of them is...,None,[We have freedom in this country one of them i...


In [51]:
#Apply
df

# THIS IS THE PART THAT ACTUALLY DOES THE SENTIMENT ANALISYS 
li = df['sentences'].tolist()
    
results = []
for i in li:
    text = Sentence(i)
    classifier.predict(text)
    results.append(text.labels)
print(results)    


[['Sentence: "As a reminder, this subreddit [is for civil discussion.](/r/politics/wiki/index#wiki_be_civil)In general, be courteous to others. Debate/discuss/argue the merits of ideas, don't attack people. Personal insults, shill or troll accusations, hate speech, any suggestion or support of harm, violence, or death, and other rule violations can result in a permanent ban. If you see comments in violation of our rules, please report them. For those who have questions regarding any media outlets being posted on this subreddit, please click [here](https://www.reddit.com/r/politics/wiki/approveddomainslist) to review our details as to our approved domains list and outlet criteria. **Special announcement:**  r/politics is currently accepting new moderator applications. If you want to help make this community a better place, consider [applying here today](https://www.reddit.com/r/politics/comments/sskg6a/rpolitics_is_looking_for_more_moderators/)!****I am a bot, and this action was perfor

In [53]:
 


df2 = pd.DataFrame(results, columns = ['Sentiment'])

df3 = pd.concat([df, df2], axis = 1) 

df3 
# dfexp= df3.to_excel(excel_writer=r"C:\Users\Van Cloui\Desktop\Code\Export\Sentiment_for_clean.xlsx")




In [ ]:
#CUT OFFF WHAT YOU NEED



new = df3["Sentiment"].str.split("NEGATIVE", n=1 expand = True)


for i in df3["Sentiment"]:
    df4= df3["Sentiment"].values.str.split("NEGATIVE")

# new = df3['sentence'].values.tolist()
# new2 = new.str.split("/", expand=True)

new



# df3.drop(columns =["score"], inplace = True)


# df4 = df3['sentence'].str.split('""/""',1, expand=True)

# making separate last name column from new data frame
# df4["score"]= new[1]
# df3


In [18]:
# df_clean =  pd.read_excel(r"C:\Users\Van Cloui\Desktop\Code\DATASETS\Sentiment anal\Sentiment_clean.xlsx")

df = df_clean


df2 = df[df.Sentiment == "POSITIVE"] 
df3 = df[df.Sentiment == "NEGATIVE"]

# df4 = df2['Score'].to_string()
# df2['Score'].replace(regex=True,inplace=True,to_replace=r'-',value=r'')
# df4['Score'].str.replace('-', '')


# df2["Score"] = df["Score"]*-1


# df4 = df3.append(df2, ignore_index=True)

# df4

# dfexp= df4.to_excel(excel_writer=r"C:\Users\Van Cloui\Desktop\Code\Export\Sentiment_final.xlsx")


